In [1]:
import pandas as pd

import sklearn
from sklearn import set_config

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import ShuffleSplit, KFold,cross_val_score, RandomizedSearchCV, GridSearchCV

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR

from sklearn.ensemble import VotingRegressor, RandomForestRegressor

from xgboost import XGBRegressor

In [3]:
df = pd.read_csv("df")

In [6]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [7]:
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,renovated,avg_price_per_sqft,sqft_living_percent
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,1340,5650,0,188.050847,0.208850
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,1690,7639,1,209.338521,0.354874
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,2720,8062,0,233.766234,0.077000
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,1360,5000,0,308.163265,0.392000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,1800,7503,0,303.571429,0.207921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21590,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,1530,1509,0,235.294118,1.352785
21591,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,1830,7200,0,173.160173,0.397385
21592,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,1020,2007,0,394.216667,0.755556
21593,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,1410,1287,0,250.000000,0.670017


In [11]:
df.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'sqft_living15',
       'sqft_lot15', 'renovated', 'avg_price_per_sqft', 'sqft_living_percent'],
      dtype='object')

In [8]:
y = df.price

In [15]:
categorical = ["waterfront", "view", "grade", "condition", "zipcode", "renovated"]
numerical = ["bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "sqft_above", "sqft_basement",
            "yr_built", "sqft_living15", "sqft_lot15", "avg_price_per_sqft", "sqft_living_percent"]

In [16]:
X = df[categorical+numerical]

In [17]:
X

,waterfront,view,grade,condition,zipcode,renovated,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,yr_built,sqft_living15,sqft_lot15,avg_price_per_sqft,sqft_living_percent
0,0,0,7,3,98178,0,3,1.00,1180,5650,1.0,1180,0,1955,1340,5650,188.050847,0.208850
1,0,0,7,3,98125,1,3,2.25,2570,7242,2.0,2170,400,1951,1690,7639,209.338521,0.354874
2,0,0,6,3,98028,0,2,1.00,770,10000,1.0,770,0,1933,2720,8062,233.766234,0.077000
3,0,0,7,5,98136,0,4,3.00,1960,5000,1.0,1050,910,1965,1360,5000,308.163265,0.392000
4,0,0,8,3,98074,0,3,2.00,1680,8080,1.0,1680,0,1987,1800,7503,303.571429,0.207921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21590,0,0,8,3,98103,0,3,2.50,1530,1131,3.0,1530,0,2009,1530,1509,235.294118,1.352785
21591,0,0,8,3,98146,0,4,2.50,2310,5813,2.0,2310,0,2014,1830,7200,173.160173,0.397385
21592,0,0,7,3,98144,0,2,0.75,1020,1350,2.0,1020,0,2009,1020,2007,394.216667,0.755556
21593,0,0,8,3,98027,0,3,2.50,1600,2388,2.0,1600,0,2004,1410,1287,250.000000,0.670017


In [24]:
num_process = make_pipeline(SimpleImputer(strategy = "mean"), StandardScaler())
cat_process = make_pipeline(SimpleImputer(strategy = "most_frequent"),
                            OneHotEncoder(sparse = False, handle_unknown = "ignore"), StandardScaler())

preprocess = make_column_transformer((num_process, numerical), (cat_process, categorical), remainder = "passthrough")

In [42]:
 sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [159]:
lr = LinearRegression()

linear = make_pipeline(preprocess, lr)
cv = KFold(n_splits = 5, shuffle = True)
linear_cv = cross_val_score(linear, X, y, cv = cv, scoring = "r2")

print(linear_cv)
print(linear_cv.mean())

[ 9.17729205e-01  9.12327312e-01  9.06236249e-01  8.99417544e-01
 -2.91288611e+20]
-5.825772214768933e+19


In [148]:
lasso = Lasso()

lasso = make_pipeline(preprocess, lasso)

cv = KFold(n_splits = 5, shuffle = True)

lasso_cv = cross_val_score(lasso, X, y, cv = cv, scoring = "r2")

print(lasso_cv)
print(lasso_cv.mean())

C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2924775160520.75, tolerance: 231808082508.73132
  model = cd_fast.enet_coordinate_descent(
C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3232163975543.1562, tolerance: 240348853400.07202
  model = cd_fast.enet_coordinate_descent(
C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2447831622352.4688, tolerance: 229537878487.4479
  model = cd_fast.enet_coordinate_descent(
C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

[0.90177786 0.91153748 0.90988754 0.90920792 0.91317514]
0.9091171890118606


C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3051826836739.7812, tolerance: 233453329798.22836
  model = cd_fast.enet_coordinate_descent(


In [165]:
rf = RandomForestRegressor(ccp_alpha = 0.2)

random_forest = make_pipeline(preprocess, rf)

cv = KFold(n_splits = 5, shuffle = True)
rf_cv = cross_val_score(random_forest, X, y, cv = cv, scoring = "r2")

print(rf_cv)
print(rf_cv.mean())

[0.98666353 0.99458835 0.9984019  0.99545494 0.99702758]
0.9944272606682576


In [145]:
random_forest

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors', 'sqft_above',
                                                   'sqft_basement', 'yr_built',
                                                   'sqft_living15',
                                                   'sqft_lot15',
                                                   'av

In [158]:
xgb = XGBRegressor()

xgb = make_pipeline(preprocess, xgb)

cv = KFold(n_splits = 5, shuffle = True)
xgb_cv = cross_val_score(xgb, X, y, cv =cv , scoring = "r2")

print(xgb_cv)
print(xgb_cv.mean())

[0.99317274 0.99565116 0.9925605  0.9970359  0.99768472]
0.9952210042109566


In [147]:
lasso_param_grid = {}

lasso_param_grid["lasso__max_iter"] = [5000, 10000]
lasso_param_grid["lasso__alpha"] = [1, 2]
lasso_param_grid["lasso__selection"] = ["cyclic", "random"]

cv = KFold(n_splits = 5, shuffle = True)

lasso_search = GridSearchCV(lasso, lasso_param_grid, cv = cv, scoring = "r2")
lasso_search.fit(X, y)

print("Best score: {}".format(lasso_search.best_score_))
print("Best parameters: {}".format(lasso_search.best_params_))

C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20799216510833.188, tolerance: 232869648716.87766
  model = cd_fast.enet_coordinate_descent(
C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 67283190301515.95, tolerance: 227914819120.89157
  model = cd_fast.enet_coordinate_descent(
C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 32622169566777.547, tolerance: 233589366985.7265
  model = cd_fast.enet_coordinate_descent(
C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWa

Best score: 0.9097920982684874
Best parameters: {'lasso__alpha': 2, 'lasso__max_iter': 5000, 'lasso__selection': 'random'}


C:\Users\caisy\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 27433264403564.72, tolerance: 291085349270.8699
  model = cd_fast.enet_coordinate_descent(


In [151]:
rf_param_grid = {}

rf_param_grid["randomforestregressor__max_depth"] = [10, 20, 40 , 80, None]
rf_param_grid["randomforestregressor__criterion"] = ["mse", "mae"]
rf_param_grid["randomforestregressor__min_samples_leaf"] = [1, 2, 4]
rf_param_grid["randomforestregressor__min_samples_split"] = [1, 2, 4]
rf_param_grid["randomforestregressor__n_estimators"] = [100, 200, 500, 1000]


cv = KFold(n_splits = 5, shuffle = True)

rf_search = RandomizedSearchCV(random_forest, rf_param_grid, cv = cv, n_iter = 10, scoring = "r2")
rf_search.fit(X, y)

print("Best score: {}".format(rf_search.best_score_))
print("Best parameters: {}".format(rf_search.best_params_))